<a href="https://colab.research.google.com/github/SGRGit/gwOsc/blob/Main/DataPreprocessingImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
%matplotlib inline
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score

In [0]:
import os
base_dir = "drive/My Drive/Colab Notebooks/Other"
seta = 'George_W_Bush'
setb = 'Collin_Powell'

data_dir = os.path.join(base_dir, 'data')
seta_data_dir = os.path.join(data_dir, seta)
seta_data_fnnames = os.listdir(seta_data_dir)

In [0]:

img_path1 = os.path.join(seta_data_dir, seta_data_fnnames[0])
img1 = load_img(img_path1, target_size = (150,150))
x1 = img_to_array(img1)
x1 = x1.reshape(1, 150, 150, 3)

img_path2 = os.path.join(seta_data_dir, seta_data_fnnames[1])
img2 = load_img(img_path2, target_size = (150,150))
x2 = img_to_array(img2)
x2 = x2.reshape(1, 150, 150, 3)

#np.vstack((x1,x2))

In [0]:
q = np.empty((1, 150, 150, 3))
for a in range(1, 2, 1):
  img_path = os.path.join(seta_data_dir, seta_data_fnnames[a])
  img = load_img(img_path, target_size = (150,150))
  img_to_array(img).reshape(1, 150, 150, 3)
  q = np.vstack((q, img_to_array(img).reshape(1, 150, 150, 3)))

IndexError: ignored